# Inference Service

## Imports

In [1]:
import os
from pathlib import Path
from typing import *

from dotenv import load_dotenv


assert load_dotenv(dotenv_path=Path("../.env"))

In [2]:
from python_utils.model_schemas.request import (
    ServiceConnectionDetails,
    get_service_connection_details,
    EndpointConfig,
)

inference_service_conn_details: ServiceConnectionDetails = (
    get_service_connection_details(prefix="INFERENCE")
)
print(f"{inference_service_conn_details=}")

embed_images_inference_service_endpoint_config = EndpointConfig(
    connection=inference_service_conn_details, endpoint="/features/images"
)
embed_images_url: str = embed_images_inference_service_endpoint_config.build_url()
print(f"{embed_images_url=}")
embed_text_inference_service_endpoint_config = EndpointConfig(
    connection=inference_service_conn_details, endpoint="/features/text"
)
embed_text_url: str = embed_text_inference_service_endpoint_config.build_url()
print(f"{embed_text_url=}")

inference_service_conn_details=ServiceConnectionDetails(protocol='http', host='localhost', port=8004, timeout=30)
embed_images_url='http://localhost:8004/features/images'
embed_text_url='http://localhost:8004/features/text'


In [3]:
import base64
from io import BytesIO

from PIL import Image


def pil_image_to_base64_string(pil_image: Image) -> str:
    """
    Convert a PIL Image object to a base64 string.

    Args:
        pil_image (Image): The PIL Image object to convert.

    Returns:
        str: The base64 string representation of the PIL Image object.
    """

    image_buffer = BytesIO()
    pil_image.save(image_buffer, format="PNG")

    image_bytes: bytes = image_buffer.getvalue()
    base64_string_image_content: str = base64.b64encode(image_bytes).decode("utf-8")

    return base64_string_image_content


cat: Image = Image.open("../demo_assets/cat.png")
dog: Image = Image.open("../demo_assets/dog.png")

In [4]:
from python_utils.model_schemas.images import ImageFile
from python_utils.model_schemas.files import (
    FileUpload,
    FileDetails,
    Base64StringContent,
)

images: List[ImageFile] = [
    ImageFile(
        file_upload=FileUpload(
            file_details=FileDetails(
                file_stem="image",
                mime_type="png",
            ),
            file_content=Base64StringContent(
                base64_string_content=pil_image_to_base64_string(pil_image)
            ),
        )
    )
    for pil_image in [cat, dog]
]
images

[ImageFile(file_upload=FileUpload(file_stem=image, mime_type=png, base64_string_content=iVBORw0KGgoAAAAN... (length=517856), , uid=None, url=None),
 ImageFile(file_upload=FileUpload(file_stem=image, mime_type=png, base64_string_content=iVBORw0KGgoAAAAN... (length=1372232), , uid=None, url=None)]

In [8]:
embed_images_response: List[List[float]] = (
    embed_images_inference_service_endpoint_config.post_json(
        json_payload=[image.model_dump(mode="json") for image in images]
    )
)
print(f"{len(embed_images_response)=}")
embed_images_response[0][:10]

len(embed_images_response)=2


[1.116468906402588,
 -1.7898609638214111,
 2.6711695194244385,
 1.9941316843032837,
 -0.08995503187179565,
 -3.0010790824890137,
 -0.9132192730903625,
 1.5771819353103638,
 3.2596993446350098,
 1.7104109525680542]

In [9]:
embed_text_response: List[List[float]] = (
    embed_text_inference_service_endpoint_config.post_list(
        list_payload=["this", "that", "the other"]
    )
)
print(f"{len(embed_text_response)=}")
embed_text_response[0][:10]

len(embed_text_response)=3


[-0.14703670144081116,
 -2.245680332183838,
 1.1033319234848022,
 1.1573079824447632,
 -2.568621873855591,
 -0.8461986780166626,
 -1.128066897392273,
 -0.4634495973587036,
 0.20879024267196655,
 1.4117908477783203]

In [11]:
os.environ["PE_MODEL_NAME"]

'PE-Core-G14-448'